# Procesamiento del Lenguaje Natural

## Tarea 02 (práctica 02)

Francisco Pablo Rodrigo

### Configuraciones previas

Se realizan los *imports* necesarios para hacer el análisis de nuestro *corpus*

In [2]:
#-*- encoding:utf-8 -*-
import nltk
# Corpus en español
from nltk.corpus import cess_esp
# Freqdist será de utilidad para medir la 
# frecuencia de los tipos
from nltk.probability import FreqDist

import numpy as np
from collections import defaultdict, Counter
from itertools import chain

# Imports relacionados con la creación de gráficas
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
# Funcion que crea un vocabulario de palabras con un indice numero
def vocab():
    vocab = defaultdict()
    vocab.default_factory = lambda: len (vocab)
    return vocab

# Funcion que pasa la cadena de simbolos a una secuencia con indices numericos
def text2numba(corpus, vocab):
    for doc in corpus:
        yield [vocab[w] for w in doc.split()]

Los comandos o instrucciones que solo se ejecutan una vez se muestran comentados. Generalmente corresponde a descargargas o instalación de paquetes

In [4]:
# Esto solo se debe ejecutar una vez para del corpus.
# Se comenta para no descargarlo varias veces por error.
#nltk.download('cess_esp')

### Elección de un corpus

Un **corpus** es una muestra bien organizada del nuestro lenguaje tomada de materiales escritos o hablados y que se encuentran agrupados bajo un críterio común.

Para esta práctica se utilizará un corpus en *español* provisto por la librería de *nltk*, en este caso utilizaremos  **cess_esp**. La recomendación es siempre utilizar un corpus con un lenguaje del que estemos bien familizarizamos, de lo contrario se tendrán problemas para poder analizarlo debido a que no se conocerán a la perfeccción los elementos léxicos y no léxicos, son *stopwords*, etc. En ese sentido debido a nuestra formación y a la semejanza entre idiomas se puede utilizar un corpus en español o inglés sin que estos suponga mucho problema.

In [5]:
# Se trabajara con el formato en palabras, 
# teniendo en cuenta que hay otros formatos como 
# Sentencias o texto
# https://www.nltk.org/api/nltk.corpus.html
corpus = cess_esp.sents()

### Limpiar el corpus: eliminar signos de puntuación, de interrogación, admiración y elementos no léxicos.

Para tener un análisis más limpio de los tipos y morfemas se descartan signos de puntuación y algunos otros elementos no léxicos que se muestran a continuación.

In [6]:
# Se declaran elementos no lexicos con base en el idioma del corpus
non_lexical_list = ['¡','!','?','¿','.',',',';',':','-','_','(',')','""',"'",
                    '-','#','/','&',
                    '$','*','"','-fpa-','-fpt-','*0*','-Fpa-','-Fpt-','-fe-']

In [7]:
# Se crea una nueva lista sin los elementos lexicos

corpus_without_non_lexical = []

for sent in corpus:
    corpus_without_non_lexical.append([w.lower() for w in sent if w not in non_lexical_list])

In [8]:
print(corpus_without_non_lexical[2])

# Separando en corpus de entrenamiento y de prueba
corpus, corpus_eval = train_test_split(corpus_without_non_lexical,test_size=0.95)
#corpus = corpus_without_non_lexical

['la', 'electricidad', 'producida', 'pasará', 'a', 'la', 'red', 'eléctrica', 'pública', 'de', 'méxico', 'en_virtud_de', 'un', 'acuerdo', 'de', 'venta', 'de', 'energía', 'de', 'eaa', 'con', 'la', 'comisión_federal_de_electricidad', 'cfe', 'por', 'una', 'duración', 'de', '25', 'años']


### 1.  Stemming

Para esta tarea no es necesario realizar el procesos de steamming

### 2. Insertar símbolos de inicio y final de cadena

Se indexa cada simbolo del vocabulario previamente tratado (sin _stopwords_ y _estemmizado_) para tener un modelo de entrenamiento.

In [9]:
# Llamamos a la funcion para crear un vocabulario
idx = vocab() # Simplemente se renombra la funcion

sents_corpus = []

for sent in corpus:
    sents_corpus.append(" ".join(sent)) # Can make it better
   

cads_idx = list(text2numba(sents_corpus,idx))
#print(cads_idx[:3])

#print(idx)

Además, se colocarán etiquetas al inicio y al final de cada sentencia: BOS (Beginning of Sentence) y EOS (End of Sentence) respectivamente.

In [10]:
BOS = '<BOS>'
EOS = '<EOS>'

# A cada etiqueta se le asigna el indice número mayor 
# que el último indice asignado al vocabulario

BOS_IDX = max(idx.values()) + 1
EOS_IDX = max(idx.values()) + 2

# Se agregan las etiquetas al vocabulario
idx[BOS] = EOS_IDX
idx[EOS] = BOS_IDX

# Agregamos las etiquetas BOS al inicio y EOS al final de cada sentencia

strings = [[BOS_IDX] + cad + [EOS_IDX] for cad in cads_idx]

print(strings[:2])

[[3242, 0, 1, 2, 3, 4, 5, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 3, 18, 3, 19, 20, 21, 22, 10, 23, 24, 14, 25, 26, 27, 22, 28, 29, 30, 3, 31, 32, 7, 33, 34, 3243], [3242, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 24, 35, 45, 46, 3, 4, 47, 48, 49, 33, 50, 51, 52, 53, 24, 54, 55, 22, 56, 24, 57, 58, 59, 60, 3, 61, 62, 3243]]


### 3. Bigramas

In [11]:
# Creacion de bigramas

bigrams = list(chain(*[zip(cad,cad[1:]) for cad in strings]))
print(len(bigrams))
# print(bigrams[:5])

8514


### 4. Entrenamiento de la red neuronal

In [18]:
np.random.seed(0)

# dim -> hiperparametro que define la dimensioón de los vectores-palabra
dim = 100
m = 3
N = len(idx)

# Se usa para generar vectores one-shot
matrix_I = np.identity(N)

# Embebidding
C = np.random.randn(dim,N) / np.sqrt(N)

# Oculta
W = np.random.randn(m,dim) / np.sqrt(dim)
b = np.ones(m)

# Salida
U = np.random.randn(N,m) / np.sqrt(m)
c = np.ones(N)

#V = np.random.randn(N,dim) / np.sqrt(dim)

iterations = 100
eta = 0.1

for i in tqdm(range(0,iterations)):    
    for bigram in bigrams:
        
        # FOWARD       
        
        # Capa embbeding
        c_i = C.T[bigram[0]]
        
        # Capa oculta
        h_i = np.tanh(np.dot(W,c_i) + b)
        
        # Pre-activacion
        a = np.dot(U,h_i) + c
        
        # Salidas
        tmp = np.exp(a - np.max(a))
        # Aplicando softmax
        f = tmp/tmp.sum(0)
        
        # BACKPROPAGATION para salida
        d_out = f
        k= bigram[1]
        d_out[k] -= 1
     
        # Backpropagation para la capa oculta
        dh = (1-h_i**2)*np.dot(U.T,d_out) 
        
        # Backpropagation para la capa embedding
        dc = np.dot(W.T,dh)
        c -= f

        # Actualizacion de la capa de salida
        U -= eta*np.outer(d_out,h_i)
        
        # Actualizacion de capa oculta
        W -= eta*np.outer(dh,c_i)
        b -=eta*h_i.sum(0)
        
        # Actualizacion embedding
        C -= eta*np.outer(dc,matrix_I[bigram[0]].T)
        break

100%|██████████| 100/100 [00:00<00:00, 710.91it/s]


In [58]:
def forward(x): 
        
    # Capa embbeding
    c_i = C.T[x]

    # Capa oculta
    h_i = np.tanh(np.dot(W,c_i) + b)

    # Pre-activacion
    a = np.dot(U,h_i) + c

    # Salidas
    tmp = np.exp(a - np.max(a))
    
    # Aplicando softmax
    f = tmp/tmp.sum(0)
    
    return f

In [82]:
lista = []

for word in idx.keys():
    lista.append((word,forward(idx['contra'])[idx[word]]))
    #print(word,forward(idx['presidente'])[idx[word]])

lista.sort(key=lambda x: x[1],reverse=True)

In [83]:
lista

[('la', 0.1271541170622606),
 ('el', 0.11908486010151934),
 ('los', 0.10071465520599743),
 ('las', 0.04367956218694011),
 ('su', 0.023828529053112394),
 ('una', 0.02285197368729347),
 ('un', 0.018117026353820374),
 ('sus', 0.006359934337405533),
 ('lo', 0.0059421205155275715),
 ('personas', 0.0054202191378497685),
 ('esta', 0.005157947451802433),
 ('ha', 0.004421304786734775),
 ('este', 0.003919937953663765),
 ('ese', 0.0034648627649422597),
 ('ser', 0.003300043844998693),
 ('nuestro', 0.003242044649200445),
 ('cada', 0.003222075481023596),
 ('eso', 0.0031103884244297487),
 ('esa', 0.0029384122576928354),
 ('oro', 0.002875701127991173),
 ('no', 0.0027587397575732854),
 ('estar', 0.002661407371183464),
 ('tan', 0.002479281732602047),
 ('haberse', 0.0023507373721071223),
 ('muchos', 0.002302845253963208),
 ('ellos', 0.002297681073960467),
 ('efe', 0.002246249138178281),
 ('hacer', 0.002206945431022796),
 ('condiciones', 0.002114531726853987),
 ('estos', 0.0021065160846111563),
 ('en', 0.